In [ ]:
from util.plot import plot_interactive_lines

## BDS (CSV Data)

### Download

https://www.census.gov/data/datasets/time-series/econ/bds/bds-datasets.html

In [ ]:
base_url = "https://www2.census.gov/programs-surveys/bds/tables/time-series/"

In [ ]:
dfe_s = pd.read_csv(base_url+"bds2018_sector.csv")

In [ ]:
dfe_az = pd.read_csv(base_url+"bds2018_eage_esize.csv")

In [ ]:
dfe_sa = pd.read_csv(base_url+"bds2018_sector_eage.csv")

### Categories

In [ ]:
cat_tables = pd.read_html("https://www.census.gov/programs-surveys/bds/documentation/glossary.html")

In [ ]:
cat_e_age, cat_e_size, cat_f_age, cat_f_size, cat_nonmetro, cat_sector, cat_state = [
    i.dropna(axis=1, how="all") for i in cat_tables
]

In [ ]:
cat_sector = cat_sector.set_index(0).iloc[1:,0].to_dict()

In [ ]:
def add_

### Explore

(experimental explore // should create clean functions here and make another notebook for exploration)

#### Entry/Exit Rate by Sector

In [ ]:
dt = dfe_s.assign(sector_ = dfe_s.sector.map(cat_sector))

In [ ]:
plot_interactive_lines(dt, "year", "estabs_entry_rate", "sector_")

In [ ]:
plot_interactive_lines(dt, "year", "estabs_exit_rate", "sector_")

#### Average Size by Sector 

In [ ]:
dt = (dfe_s
         .eval("avgsize_e = emp / estabs")
         .eval("avgsize_f = emp / firms")
         .assign(sector_ = dfe_s.sector.map(cat_sector))
        )

In [ ]:
plot_interactive_lines(dt, "year", "avgsize_e", "sector_")

In [ ]:
plot_interactive_lines(dt, "year", "avgsize_f", "sector_")

#### Entry Size By Sector

In [ ]:
entry_age = "a) 0"
dt = (dfe_sa
         .eval("avgsize_e = emp / estabs")
         .eval("avgsize_f = emp / firms")
         .assign(sector_ = dfe_sa.sector.map(cat_sector))
         .query('eage == @entry_age')
        )

In [ ]:
plot_interactive_lines(dt, "year", "avgsize_e", "sector_")

In [ ]:
plot_interactive_lines(dt, "year", "avgsize_f", "sector_")